# Imports

In [1]:
# Imports
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread

import tensorflow.image as tf_image
from tensorflow.keras import layers, models, callbacks, Model

2022-12-07 00:02:38.526377: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-07 00:02:38.526394: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from nltk.corpus import stopwords 
import string
from nltk import word_tokenize 
# import nltk
from gensim.models import Word2Vec
# from sklearn.model_selection import train_test_split
import gensim.downloader as api
from keras.preprocessing.sequence import pad_sequences
nltk.download('stopwords')

NameError: name 'nltk' is not defined

# Paths and key frames

In [ ]:
# Paths

rel_imagedir_path = '../../../data/screenshots/'
legit_image_path = os.path.join(rel_imagedir_path, 'legit_screenshots')
scam_image_path = os.path.join(rel_imagedir_path, 'scam_screenshots')

In [ ]:
legit_image_path, scam_image_path

In [ ]:
# Legit websites

legit_key_df = pd.read_csv('legim_text_screenshots_key.csv')
print(legit_key_df.shape)

In [ ]:
legit_key_df = legit_key_df.drop(columns=['Unnamed: 0', 'index'])
legit_key_df['target'] = 0
legit_key_df = legit_key_df.drop_duplicates(subset='url')
legit_key_df

In [ ]:
# Scam websites

scam_key_df = pd.read_csv('scam_text_screenshots_key.csv')
scam_key_df = scam_key_df.drop(columns=['Unnamed: 0'])
scam_key_df['target'] = 1
scam_key_df

In [ ]:
merged_key_df = pd.concat([legit_key_df, scam_key_df], ignore_index=True).reset_index(drop=True)
merged_key_df = merged_key_df.sample(frac=1).reset_index(drop=True)

In [ ]:
merged_key_df

# Checking image shape

In [ ]:
def image_shape_check(image_dir):
    
    print('Generating Paths')
    image_paths = []
    for image in os.listdir(image_dir):
        if image.endswith('.png'):
            # print(image)
            image_paths.append(os.path.join(image_dir, image))
    # print(image_paths)
    
    print('Checking Shapes')
    image_shapes = []
    for image in image_paths:
        np_image = imread(image)
        image_shapes.append(np_image.shape)
    
    print(f'Shapes found: {list(set(image_shapes))}')

In [ ]:
# Takes a while to run and will use a lot of memory
# image_shape_check(legit_image_path)

In [ ]:
# Takes a while to run and will use a lot of memory
# image_shape_check(scam_image_path)

In [ ]:
# def pillow_check(path_, img='3.png'):
#     image = Image.open(os.path.join(path_, img))
#     print(f'Image format: {image.format}')
#     print(f'Image size: {image.size}')
#     print(f'Image channels: {image.mode}')

In [ ]:
# pillow_check(legit_image_path, '5.png')

In [ ]:
# pillow_check(scam_image_path, '5.png')

# Loading Data

In [ ]:
def load_data(key_df, legit_imagedir_path, scam_imagedir_path):
    
    X_image = []
    X_text = []
    y = []
    
    for index, row in key_df.iterrows():
        if index % 500 == 0:
            print(f'loaded {index} data points')
        
        # Append image to X_image
        if row['target'] == 0:
            image_path = os.path.join(legit_imagedir_path, f"{row['num_of_picture']}.png")
            np_legit_image = imread(image_path)[:, :, :3] # slice off the alpha channel
            np_legit_image = tf_image.resize(np_legit_image, (300, 400))
            X_image.append(np_legit_image)
            
        elif row['target'] == 1:
            image_path = os.path.join(scam_imagedir_path, f"{row['num_of_picture']}.png")
            np_legit_image = imread(image_path)[:, :, :3] # slice off the alpha channel
            np_legit_image = tf_image.resize(np_legit_image, (300, 400))
            X_image.append(np_legit_image)
        
        # Load text
        X_text.append(row['text'])
        
        y.append(row['target'])
    
    print('\nFinished loading data!')
    # print(X_image)
    return np.array(X_image), X_text, np.array(y)

In [ ]:
merged_key_df

In [ ]:
X_image, X_text, y = load_data(merged_key_df, legit_image_path, scam_image_path)

# Proproccess Text

In [ ]:
# Helper Functions

def clean(text):
    text = text.split()
    words_only = [word for word in text if word.isalpha()]
    for punctuation in string.punctuation:
        words_only = [word.replace(punctuation, ' ').lower() for word in words_only] # Remove Punctuation
    # lowercased = text.lower() # Lower Case
    #tokenized = word_tokenize(lowercased) # Tokenize
    # words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    # lemma=WordNetLemmatizer() # Initiate Lemmatizer
    # lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return without_stopwords

def text_cleaner(list_text):
    print('Cleaning text')
    list_clean_text=[]
    for text in list_text:
        clean_txt=clean(text) # Use clean function
        list_clean_text.append(clean_txt)

    return list_clean_text

def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return embedded_sentence

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    print('Embedding with word2vec')
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

def preprocess_pad(list_text):
    # Clean - remove punctuation and stopwords
    list_ = text_cleaner(list_text)
    
    # Embed with word2vec
    word2vec_transfer = api.load('glove-wiki-gigaword-100')
    list_ = embedding(word2vec_transfer, list_)
    
    # Pad Sequences
    print('Padding sequences')
    list_ = pad_sequences(list_, dtype='float32', padding='post', maxlen=200)
    
    print('Text preprocessing and embedding complete!')
    return list_

In [ ]:
# def preprocess_pad(list_text):
#     df_clean=text_cleaner(X_text)
#     df_clean.reset_index(drop = True, inplace=True)
#     word2vec_transfer = api.load('glove-wiki-gigaword-100')
#     X_train_embed_2 = embedding(word2vec_transfer, df_clean['clean_text'])
#     X_train_pad = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
#     return X_train_pad

In [ ]:
X_text = preprocess_pad(X_text)

# Functional Model

In [ ]:
from tensorflow.keras import layers, models, callbacks, Model

def build_model():
    
    # CNN Architecture
    cnn_input = layers.Input(shape=X_image.shape[1:])

    x = layers.Conv2D(16, (4, 4), activation='relu')(cnn_input)
    x = layers.MaxPool2D(2, 2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(16, (4, 4), activation='relu')(x)
    x = layers.MaxPool2D(2, 2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(16, (4, 4), activation='relu')(x)
    x = layers.MaxPool2D(2, 2)(x)
    x = layers.BatchNormalization()(x)
    
    cnn_output = layers.Flatten()(x)
    
    # LSTM Architecture
    nlp_input =layers.Input(shape=(200,100))
    y = layers.Masking()(nlp_input)
    y = layers.BatchNormalization()(y)

    y = layers.Bidirectional(layers.LSTM(32, activation='tanh', return_sequences=True))(y)
    y = layers.Dropout(0.5)(y)

    y = layers.Bidirectional(layers.LSTM(32, activation='tanh', return_sequences=True))(y)
    y = layers.Dropout(0.5)(y)

    y = layers.Bidirectional(layers.LSTM(32, activation='tanh', return_sequences=False))(y)
    nlp_output = layers.Dropout(0.5)(y)
    
    # Define NLP model and concatenate output
    combined = layers.concatenate([cnn_output, nlp_output])
    
    z = layers.Dense(64, activation='relu')(combined)
    z = layers.Dropout(0.4)(z)
    z = layers.Dense(32, activation='relu')(z)
    z = layers.Dropout(0.2)(z)
    
    final_output = layers.Dense(1, activation='sigmoid')(z)

    model = Model(inputs=[cnn_input, nlp_input],outputs=final_output)
    
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    (X_image, X_text_pad),
    y,
    epochs=50,
    batch_size=16,
    validation_split=0.3,
    callbacks=callbacks.EarlyStopping(patience=5, restore_best_weights=True)
)